In [ ]:
! pip install -U langchain openai langchain-experimental # (newest versions required for multi-modal)
! pip install "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib chromadb tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: pa

In [ ]:
#!pip install unstructured

In [6]:
!pip show unstructured

Name: unstructured
Version: 0.14.0
Summary: A library that prepares raw documents for downstream ML tasks.
Home-page: https://github.com/Unstructured-IO/unstructured
Author: Unstructured Technologies
Author-email: devops@unstructuredai.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: backoff, beautifulsoup4, chardet, dataclasses-json, emoji, filetype, langdetect, lxml, nltk, numpy, python-iso639, python-magic, rapidfuzz, requests, tabulate, typing-extensions, unstructured-client, wrapt
Required-by: 


# Readme

- This notebook use the azure openai gpt, Remember to change your azure api key, endpoint and model_llm, folder_path.

- In my development, I totally have 5 chapter from CFA textbook, each chapter file is in pdf version. We need to extract the text from each chapter to generate the question. 

- You can change the prompt based on different situation, tis notebook is for CPA. So the role here is "You are the CFA equity Investment examiner"


In [192]:
import glob
import pandas as pd
from langchain.document_loaders import UnstructuredFileLoader
from openai import AzureOpenAI
import time

folder_path="/content/"
api_key = "<API_KEY>"
azure_endpoint = "<Endpoint>"
model_llm="test"


number_chunck_selected_for_each_file=100


def get_pdf_file_paths(folder_path):
    
    """
    get the pdf file paths in the folder

    Args:
      folder_path: the path of the folder that contains the pdf files
    """
    pdf_files = glob.glob(folder_path + "/*.pdf")
    return pdf_files



def pdf_to_chunck_list(result1,path):
  """
  convert the pdf file to the list of chunck

  Args:
    path: the path of the pdf file
    result1: the list of chunck that will be appended to the result
  """
  result=[]
  loader = UnstructuredFileLoader(
    path, mode="elements"
  )
  docs = loader.load()

  for i in range(len(docs)):
    if docs[i].metadata["category"] in ["UncategorizedText","Footer","Header","Title"]:
      pass

    else:
      result.append(docs[i].page_content)

  # if len(result)>number_chunck_selected_for_each_file:
  #   result=result[:101]

  for i in range(len(result)):
    result1.append(result[i])

  return result1


def prepare_dataframe(folder_path):
  """
  prepare the data to put into the chatgpt

  Args:
    folder_path: the path of the folder that contains the pdf files
  """
  pdf_file_paths = get_pdf_file_paths(folder_path)
  result=[]
  for i in pdf_file_paths:
    result=pdf_to_chunck_list(result,i)

  df=pd.DataFrame({"input":result})


  ## Data Cleaning - delete the NULL
  empty_mask = df["input"] == ""
  df = df[~empty_mask]
  empty_mask = df["input"].isnull()
  df=df[~empty_mask]
  empty_mask = df["input"].isna()
  df=df[~empty_mask]
  filtered_rows = df[df['input'].str.split(" ").str.len() < 20]
  df = df.drop(filtered_rows.index)
  df = df.reset_index(drop=True)



  return df




def question_generate_from_llm(df):
    
    """
    use the chatgpt to generate the question based on the answer

    Args:
      df: the dataframe that contains the answer
    
    """
    error = None
    client = AzureOpenAI(
        api_key=api_key,
        api_version="2023-12-01-preview",
        azure_endpoint=azure_endpoint
    )
    result = []
    token_count = 0
    try:
        for index, row in df.iterrows():
            prompt = "<text>" + row['input'] + "</text><task>this text is the exam question answer, you need to generate the question based on it </task>"
            message = [
                {"role": "system", "content": "you are the CFA equity Investment examiner"},
                {"role": "user", "content": prompt}
            ]
            token_count = len(prompt.split(" ")) + token_count

            if token_count > 3000:
                time.sleep(5)
                token_count = 0

            while True:
                try:
                    completion = client.chat.completions.create(
                        model=model_llm,
                        messages=message
                    )
                    result.append(completion.choices[0].message.content)
                    print("current index:", index)
                    break
                except Exception as e:
                    error = e
                    # Handle rate limit exceeded error
                    if "code" in e.error or e.error["code"] == 429:
                        # Sleep for 60 seconds and retry after rate limit reset
                        time.sleep(60)
                        print("Error:", e)
                        print("Current index:", index)
                        break
                    else:
                        # Handle other exceptions
                        print("Error:", e)
                        break

            if error is not None:
                print("Current index of the DataFrame is:", index)
    except KeyboardInterrupt:
        print("Execution interrupted.")

    df=pd.DataFrame({"result":result})
    df.to_csv("backup.csv")

    return result

def merge_llm_result_to_dataframe(result,df):

  """
  merge the result from the chatgpt to the dataframe

  Args:
    result: the result from the chatgpt
    df: the dataframe that contains the answer
  
  """
  df.rename(columns={'input': 'output'}, inplace=True)
  df['input']=result
  return df



# if __name__ == '__main__':
#   df=prepare_dataframe(folder_path)
#   print("finish prepare datafraome")
#   result=question_generate_from_llm(df)
#   print("finish generate from llm")
#   length=len(result)
#   df=merge_llm_result_to_dataframe(result,df[:length])
#   df.to_csv("dataset.csv")








In [189]:
df=prepare_dataframe(folder_path)
data=question_generate_from_llm(df)
data_output=merge_llm_result_to_dataframe(data,df)
data_output.to_csv("result.csv")

In [171]:
# error=None
# api_key="ebc76beedf89486382e773d3b4cf0b10"

# client = AzureOpenAI(
#       api_key=api_key,
#       api_version="2023-12-01-preview",
#       azure_endpoint="https://gpgpt.openai.azure.com/"
#   )

In [182]:
# for i in range(5):
#   prompt = "<text>" + df.iloc[i]['input'] + "</text><task>this text is the exam question answer, you need to generate the question based on it </task>"
#   message = [
#                 {"role": "system", "content": "you are the CFA equity Investment examiner"},
#                 {"role": "user", "content": prompt}
#             ]
#   completion = client.chat.completions.create(
#                       model="test",
#                       messages=message
#                   )
#   print(completion.choices[0].message.content)





What are the functions that entities can perform in the financial markets?
What are the two main tasks of a CFA equity investment examiner?
What are the types of financial assets and real assets?
What is the difference between public securities and private securities in terms of trading, regulatory oversight, and liquidity?
What is the basis for the value of financial derivative contracts, and how do they differ from physical derivative contracts?


'2. Determine the returns (i.e., interest rates) that equate the total supply of savings with the total demand for borrowing. 3. Allocate capital to its most efficient uses.'

['What are the three main functions of the financial system?']

In [120]:
# length=len(data)
# data_copy=data
# data_output=merge_llm_result_to_dataframe(data_copy,df[:length])
# data_output

<ipython-input-116-632883253dc1>:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'input': 'output'}, inplace=True)
<ipython-input-116-632883253dc1>:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['input']=result


,output,input
0,The 2024 spring wage offensive points toward s...,What is the expected real wage growth and how ...
1,Strategy: Based on our analysis of past perfor...,What is JP Morgan's outlook on the Japanese do...
3,Cosmetics and personal care: Any increase in c...,Which Japanese companies in the cosmetics and ...
4,Transportation and leisure services: We look f...,What subsectors within transportation and leis...
5,Source: J.P. Morgan Note: Focus stocks (upside...,What is the date of the ratings mentioned in t...
...,...,...
96,Potential Investment Banking Compensation: J.P...,What is the compensation that J.P. Morgan expe...
97,Debt Position: J.P. Morgan may hold a position...,What companies' debt securities does J.P. Morg...
98,Company-Specific Disclosures: Important disclo...,What type of disclosures are available for Com...
99,"reports and all J.P. Morgan–covered companies,...",What are the ways to access reports on J.P. Mo...


In [133]:
# # data_output=data_output[['input','output']]
# filtered_rows = data_output[data_output['output'].astype(str).str.contains('disclose|Source', case=False)]
# data_output = data_output.drop(filtered_rows.index)
# data_output.to_csv("data_output_result.csv")

In [135]:
# filtered_rows = data_output[data_output['output'].str.split(" ").str.len() < 20]
# data_output = data_output.drop(filtered_rows.index)
# data_output.to_csv("data_output_result.csv")

In [136]:
# data_output = data_output.reset_index(drop=True)
# data_output.to_csv("data_output_result.csv")

In [128]:
# filtered_rows.to_csv("check.csv")

In [154]:
# loader = UnstructuredFileLoader(
#     '/content/equity_Investment.pdf', mode="elements"
#   )
# docs = loader.load()
# result1=[]
# result2=[]
# for i in range(len(docs)):
#   result1.append(docs[i].metadata["category"])
#   result2.append(docs[i].page_content)



In [155]:
# result=pd.DataFrame({'category':result1,'text':result2})

In [156]:
# result["category"].value_counts()


category
NarrativeText        232
Title                 32
UncategorizedText      9
ListItem               7
Name: count, dtype: int64

In [160]:
# result_filtered = result[result['category'] == 'NarrativeText']
# result_filtered.to_csv('result.csv')